# More SQL parsing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mo_sql_parsing as mosp

## Basics, predicates

In [3]:
mosp.parse("SELECT * FROM foo, bar WHERE id = fkey")

{'select': '*', 'from': ['foo', 'bar'], 'where': {'eq': ['id', 'fkey']}}

In [4]:
mosp.parse("SELECT * FROM foo, bar WHERE id != fkey")

{'select': '*', 'from': ['foo', 'bar'], 'where': {'neq': ['id', 'fkey']}}

In [5]:
mosp.parse("SELECT * FROM foo WHERE id = 'bar'")

{'select': '*', 'from': 'foo', 'where': {'eq': ['id', {'literal': 'bar'}]}}

In [6]:
mosp.parse("SELECT * FROM foo WHERE id LIKE 'bar'")

{'select': '*', 'from': 'foo', 'where': {'like': ['id', {'literal': 'bar'}]}}

In [7]:
mosp.parse("SELECT * FROM foo WHERE id NOT LIKE 'bar'")

{'select': '*',
 'from': 'foo',
 'where': {'not_like': ['id', {'literal': 'bar'}]}}

In [8]:
mosp.parse("SELECT * FROM foo WHERE id < '1990-01-10'")

{'select': '*',
 'from': 'foo',
 'where': {'lt': ['id', {'literal': '1990-01-10'}]}}

In [9]:
mosp.parse("SELECT * FROM foo, bar WHERE id IN ('bar', 'baz')")

{'select': '*',
 'from': ['foo', 'bar'],
 'where': {'in': ['id', {'literal': ['bar', 'baz']}]}}

In [10]:
mosp.parse("SELECT * FROM foo WHERE id = 42")

{'select': '*', 'from': 'foo', 'where': {'eq': ['id', 42]}}

## Aliases

In [11]:
mosp.parse("SELECT * FROM foo f, bar b WHERE f.id = b.fkey")

{'select': '*',
 'from': [{'value': 'foo', 'name': 'f'}, {'value': 'bar', 'name': 'b'}],
 'where': {'eq': ['f.id', 'b.fkey']}}

## Subqueries

In [12]:
mosp.parse("SELECT * FROM foo WHERE id IN (SELECT fkey FROM bar)")

{'select': '*',
 'from': 'foo',
 'where': {'in': ['id', {'select': {'value': 'fkey'}, 'from': 'bar'}]}}

## Special predicates

In [13]:
mosp.parse("SELECT * FROM foo WHERE id IS NOT NULL")

{'select': '*', 'from': 'foo', 'where': {'exists': 'id'}}

In [14]:
mosp.parse("SELECT * FROM foo WHERE id IS NULL")

{'select': '*', 'from': 'foo', 'where': {'missing': 'id'}}

## Advanced value manipulations

In [15]:
mosp.parse("SELECT * FROM foo WHERE id = CURRENT_DATE()")

{'select': '*', 'from': 'foo', 'where': {'eq': ['id', {'current_date': {}}]}}

In [16]:
mosp.parse("SELECT * FROM foo WHERE id + '5 days'::interval = 42")

{'select': '*',
 'from': 'foo',
 'where': {'eq': [{'add': ['id',
     {'cast': [{'literal': '5 days'}, {'interval': {}}]}]},
   42]}}

## SELECT ... FROM ... JOIN ...

In [17]:
mosp.parse("SELECT * FROM foo JOIN bar")

{'select': '*', 'from': ['foo', {'join': 'bar'}]}

In [18]:
mosp.parse("SELECT * FROM foo JOIN bar ON id = fkey")

{'select': '*', 'from': ['foo', {'join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [19]:
mosp.parse("SELECT * FROM foo JOIN bar ON id = fkey AND id = 'baz'")

{'select': '*',
 'from': ['foo',
  {'join': 'bar',
   'on': {'and': [{'eq': ['id', 'fkey']},
     {'eq': ['id', {'literal': 'baz'}]}]}}]}

In [20]:
mosp.parse("SELECT * FROM foo JOIN (SELECT * FROM bar) bar2 ON id = fkey")

{'select': '*',
 'from': ['foo',
  {'join': {'value': {'select': '*', 'from': 'bar'}, 'name': 'bar2'},
   'on': {'eq': ['id', 'fkey']}}]}

## Unsupported